In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from timeit import default_timer as timer

from datetime import date, timedelta

In [2]:
epi_dir = '/Volumes/umms-esnitkin/Project_KPC_LTACH/Analysis/LTACH_transmission_modeling/data'

Goal: augment the floor and room trace data with patients who were never infected

In [3]:
df_census = pd.read_excel(f"{epi_dir}/2019-02-25_KPCLTACH_C_Census.xlsx")
df_census = df_census.sort_values(["WGS_ID", "From_Date"])

In [4]:
df_census.head()

,WGS_ID,WGS_StudyID,Room,Bed,Admit_Date,DischargeDate,LOS,Room_Bed,From_Date,Thru_Date
5,C-1,1,DE/DE.SCU,1,2012-02-22,2012-10-23,244,DE.318-A,2012-06-18,2012-06-25
7,C-1,1,DE/DE.SCU,1,2012-02-22,2012-10-23,244,DE.111-B,2012-06-26,2012-07-04
3,C-1,1,DE/DE.SCU,1,2012-02-22,2012-10-23,244,DE.109-B,2012-07-05,2012-07-07
4,C-1,1,DE/DE.SCU,1,2012-02-22,2012-10-23,244,DE.117-B,2012-07-08,2012-07-23
8,C-1,1,DE/DE.SCU,1,2012-02-22,2012-10-23,244,DE.SCU8-008,2012-07-24,2012-09-17


In [6]:
df_ftrace = pd.read_csv(f"{epi_dir}/2019-12-18_facility_trace.csv", index_col=0)
df_ftrace.columns = np.arange(367)

# floor and room traces
df_floor = pd.read_csv(f"{epi_dir}/2019-12-18_floor_trace.csv", index_col=0)
df_room = pd.read_csv(f"{epi_dir}/2019-12-18_room_trace.csv", index_col=0)

In [7]:
def floor_encoder(room):
    if room in ('TMPLOAROOM-1', 'DE.TMP-1'): 
        return 5
    code = room[3]
    if code == "S": 
        return 6
    else:
        return int(code)

In [8]:
def room_encoder(room):

    return room_code

In [142]:
room.replace(r"-[AB]", "")

'DE.237-A'

'DE.237'

In [143]:
import re

In [151]:
T = (df_census["From_Date"].max() - df_census["Thru_Date"].min()).days + 1
patient_ids = np.sort(df_census["WGS_StudyID"].unique())
floor_trace = pd.DataFrame(index=patient_ids, columns=range(T))
room_trace = pd.DataFrame(index=patient_ids, columns=range(T))

min_date = df_census["Thru_Date"].min()

room_lookup = dict()
R = 1
for n in patient_ids:
    stays = df_census[df_census["WGS_StudyID"] == n]
    t = 0
    s = 0
    while (t < T):
        while s < len(stays):
            if t >= T:
                break
            from_date = (stays.iloc[s]["From_Date"] - min_date).days
            thru_date = (stays.iloc[s]["Thru_Date"] - min_date).days
            room = stays.iloc[s]["Room_Bed"]
            room = re.sub("-[AB]", "", room)
            floor_code = floor_encoder(room)
            r = room_lookup.get(room)
            if r is None:
                room_code = R
                room_lookup[room] = R
                R += 1
                if R == 8: # skip 8
                    R += 1
            else:
                room_code = r
            if t < from_date:
                floor_trace.loc[n, t] = 0
                room_trace.loc[n, t] = 0
            elif t <= thru_date:
                floor_trace.loc[n, t] = floor_code
                room_trace.loc[n, t] = room_code
            else:
                s += 1
                continue
            t += 1
        if t >= T:
            break
        floor_trace.loc[n, t] = 0
        room_trace.loc[n, t] = 0
        t += 1

In [166]:
for j in df_floor.index:
    A = df_room.loc[j].values
    B = room_trace.loc[j].values
    if not (A == B).all():
        assert ((A != 0) <= (B != 0)).all()
        if not ((A != 0) == (B != 0)).all():
            print(j)
        else:
            print(f"woo {j}")

38
65
84
85
101
129
woo 141
woo 144
149
161
172
173
183
woo 189
woo 196
197
woo 199
218
223
248
woo 252
woo 259
264
woo 295


In [177]:
df_room.loc[196].values, room_trace.loc[196].values

(array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0, 94,  5,  5,  5,  5,  5,  5,  5,  5,  5, 72, 44, 44, 44, 44, 44,
        46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0, 72, 72,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0, 

In [160]:
z = np.unique(np.array(df_room))

In [164]:
for j in df_floor.index:
    A = df_floor.loc[j].values
    B = floor_trace.loc[j].values
    if not (A == B).all():
        assert ((A != 0) <= (B != 0)).all()
        print(j)
        

38
65
84
85
101
129
149
161
172
173
183
197
199
218
223
248
264


In [116]:
df_census[df_census["WGS_StudyID"] == 38]

,WGS_ID,WGS_StudyID,Room,Bed,Admit_Date,DischargeDate,LOS,Room_Bed,From_Date,Thru_Date
492,C-38,38,DE/DE.3EAST,B,2012-05-23,2012-07-23,61,DE.305-B,2012-06-18,2012-07-22
494,C-38,38,DE/DE.3EAST,B,2012-08-10,2012-09-06,27,DE.325-B,2012-08-10,2012-08-13
493,C-38,38,DE/DE.3EAST,B,2012-08-10,2012-09-06,27,DE.301-B,2012-08-14,2012-09-05
